In [3]:
# CELL 1: Mount Google Drive
# ===============================
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# CELL 2: Install YOLOv5
# ===============================
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5/yolov5/yolov5


In [5]:

# CELL 3: Check GPU
# ===============================
!nvidia-smi
import torch
print(f"CUDA available: {torch.cuda.is_available()}")


Sun Aug 10 20:34:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8             11W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:

# CELL 4: Extract Dataset
# ===============================
import os
DATASET_ZIP_PATH = '/content/MyDrive/corrosion_images.zip'

!unzip {DATASET_ZIP_PATH} -d /content/
print("✅ Dataset extracted!")


unzip:  cannot find or open /content/MyDrive/corrosion_images.zip, /content/MyDrive/corrosion_images.zip.zip or /content/MyDrive/corrosion_images.zip.ZIP.
✅ Dataset extracted!


In [7]:

# CELL 5: Find and Check Dataset
# ===============================
!ls /content/

# Check for train folder and data.yaml
if os.path.exists('/content/data.yaml'):
    DATA_YAML_PATH = '/content/data.yaml'
    print("✅ Found data.yaml in root")
    !cat /content/data.yaml
elif os.path.exists('/content/train'):
    print("✅ Found train folder")
    !ls /content/train/

    # Check if data.yaml is inside train folder
    if os.path.exists('/content/train/data.yaml'):
        DATA_YAML_PATH = '/content/train/data.yaml'
        print("✅ Found data.yaml in train folder")
        !cat /content/train/data.yaml
    else:
        print("❌ No data.yaml found - need to create one")


data.yaml  README.dataset.txt	sample_data  yolov5
drive	   README.roboflow.txt	train
✅ Found data.yaml in root
train: ../train/images
val: ../valid/images
test: ../test/images

nc: 1
names: ['corrosion']

roboflow:
  workspace: pipeline-images
  project: object-stuff-lyr5d
  version: 1
  license: CC BY 4.0
  url: https://universe.roboflow.com/pipeline-images/object-stuff-lyr5d/dataset/1

In [9]:

# CELL 6: Create data.yaml (if needed)
# ===============================
# Only run this cell if data.yaml wasn't found above
import yaml

data_config = {
    'train': '/content/train/images',
    'val': '/content/train/images',  # Using same folder for now
    'nc': 1,  # Update: number of classes
    'names': ['corrosion']  # Update: your class names
}

with open('/content/data.yaml', 'w') as f:
    yaml.dump(data_config, f)

DATA_YAML_PATH = '/content/data.yaml'
print("✅ Created data.yaml!")
!cat /content/data.yaml


✅ Created data.yaml!
names:
- corrosion
nc: 1
train: /content/train/images
val: /content/train/images


In [10]:

# CELL 7: Split Dataset (train/val)
# ===============================
import shutil
from sklearn.model_selection import train_test_split
import glob

# Get all image files
image_files = glob.glob('/content/train/images/*.jpg') + glob.glob('/content/train/images/*.png')
print(f"📊 Total images found: {len(image_files)}")

# Split 80% train, 20% validation
train_imgs, val_imgs = train_test_split(image_files, test_size=0.2, random_state=42)

# Create validation directories
os.makedirs('/content/valid/images', exist_ok=True)
os.makedirs('/content/valid/labels', exist_ok=True)

# Move validation images and labels
for img_path in val_imgs:
    # Move image
    img_name = os.path.basename(img_path)
    shutil.move(img_path, f'/content/valid/images/{img_name}')

    # Move corresponding label
    label_name = img_name.replace('.jpg', '.txt').replace('.png', '.txt')
    label_path = f'/content/train/labels/{label_name}'
    if os.path.exists(label_path):
        shutil.move(label_path, f'/content/valid/labels/{label_name}')

print(f"✅ Split complete! Train: {len(train_imgs)}, Val: {len(val_imgs)}")

# Update data.yaml with validation path
data_config['val'] = '/content/valid/images'
with open('/content/data.yaml', 'w') as f:
    yaml.dump(data_config, f)

print("✅ Updated data.yaml with validation split")
!cat /content/data.yaml

📊 Total images found: 556
✅ Split complete! Train: 444, Val: 112
✅ Updated data.yaml with validation split
names:
- corrosion
nc: 1
train: /content/train/images
val: /content/valid/images


In [11]:


# CELL 8: Start Training
# ===============================
!python train.py \
  --img 640 \
  --batch 8 \
  --epochs 100 \
  --data /content/data.yaml \
  --weights yolov5s.pt \
  --hyp data/hyps/hyp.scratch-high.yaml \
  --project /content/drive/MyDrive/yolo_runs \
  --name corrosion_model \
  --save-period 10

print("🎉 Training completed!")


Streaming output truncated to the last 5000 lines.
      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
  0% 0/56 [00:00<?, ?it/s]/content/yolov5/yolov5/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      59/99       2.5G    0.03626    0.01217          0         17        640:   2% 1/56 [00:00<00:06,  8.35it/s]/content/yolov5/yolov5/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      59/99       2.5G    0.04377    0.01257          0         17        640:   4% 2/56 [00:00<00:07,  6.92it/s]/content/yolov5/yolov5/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      5

In [14]:

# CELL 9: Validate Model
# ===============================
BEST_WEIGHTS_PATH = '/content/drive/MyDrive/yolo_runs/corrosion_model2/weights/best.pt'

!python val.py \
  --weights {BEST_WEIGHTS_PATH} \
  --data /content/data.yaml \
  --img 640

print("📊 Validation completed!")


val: data=/content/data.yaml, weights=['/content/drive/MyDrive/yolo_runs/corrosion_model2/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-423-g567c6646 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/valid/labels.cache... 252 images, 101 backgrounds, 0 corrupt: 100% 252/252 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 8/8 [00:06<00:00,  1.20it/s]
                   all        252        237      0.577      0.523       0.53      0.322
Speed: 0.6ms pre-process, 7.8ms inference, 3.6ms NMS per image at shape (32, 3, 640, 640)
Results 

In [15]:

# CELL 10: Save Model to Google Drive
# ===============================
import shutil
from datetime import datetime

# Create model directory in Google Drive
MODEL_DIR = '/content/drive/MyDrive/trained_models'
os.makedirs(MODEL_DIR, exist_ok=True)

# Copy best model
shutil.copy2(BEST_WEIGHTS_PATH, f'{MODEL_DIR}/corrosion_detector.pt')

print(f"✅ Model saved to: {MODEL_DIR}/corrosion_detector.pt")
print("🎯 You can now download this file from Google Drive!")

✅ Model saved to: /content/drive/MyDrive/trained_models/corrosion_detector.pt
🎯 You can now download this file from Google Drive!
